In [3]:
import datetime

now = datetime.datetime.now()
print("Current date and time:", now)


Current date and time: 2025-06-30 13:18:08.732220


In [2]:
import pandas as pd
from tabulate import tabulate

# Load the results CSV
csv_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\results.csv"
df = pd.read_csv(csv_path)

# Map short model names to full names
model_name_map = {
    "RF": "RF - Random Forest",
    "GBRT": "GBRT - Gradient Boosting",
    "XGB": "XGB - XGBoost",
    "LR": "LR - Linear Regression",
    "PR": "PR - Polynomial Regression"
}
df["Model"] = df["Model"].map(model_name_map)

# Find best model per river based on lowest MAPE_Test
best_models = df.loc[df.groupby("River_Name")["MAPE_Test"].idxmin()].copy()

# Sort for readability
best_models = best_models.sort_values(by="System")

# Select and round relevant columns (now with Feature_Names)
columns_to_show = [
    "Model", "River_Name", "R2_Test", "MSE_Test", "MAPE_Test",
    "R2_Train", "MSE_Train", "MAPE_Train",
    "System", "TopK_Features", "Additional_Features_Used", "ARIMA_Enabled",
    "Feature_Names"  
]
best_models_formatted = best_models[columns_to_show].copy()


float_cols = ["R2_Test", "MSE_Test", "MAPE_Test", "R2_Train", "MSE_Train", "MAPE_Train"]
best_models_formatted[float_cols] = best_models_formatted[float_cols].applymap(lambda x: round(x, 4))

# Print table
print(tabulate(best_models_formatted, headers="keys", tablefmt="github", showindex=False))


| Model                      | River_Name            |   R2_Test |    MSE_Test |   MAPE_Test |   R2_Train |   MSE_Train |   MAPE_Train | System         |   TopK_Features | Additional_Features_Used   | ARIMA_Enabled   | Feature_Names                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_35248\215612361.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_models_formatted[float_cols] = best_models_formatted[float_cols].applymap(lambda x: round(x, 4))
